In [1]:
#pip install pygame

In [2]:
import pygame as py
import sys
import random

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame as py
import sys
import random

def drawfloor():
    screen.blit(floor_surface,(floor_x_pos,900))
    screen.blit(floor_surface,(floor_x_pos+576,900))
    
def create_pipe():
    random_pipe_pos= random.choice(pipe_height)
    bottom_pipe=pipe_surface.get_rect(midtop= (700,random_pipe_pos))
    top_pipe=pipe_surface.get_rect(midbottom= (700,random_pipe_pos-300))
    return bottom_pipe,top_pipe

def move_pipes(pipes):
    for pipe in pipes:
        pipe.centerx -=5
    return pipes

def draw_pipes(pipes):
    for pipe in pipes:
        if pipe.bottom >=1024:
            screen.blit(pipe_surface,pipe)
        else:
            flip_pipe= py.transform.flip(pipe_surface,False,True)
            screen.blit(flip_pipe,pipe)

def check_collision(pipes):
    for pipe in pipes:
        if bird_rect.colliderect(pipe):
            death_sound.play()
            return False
    if(bird_rect.top<= -100) or bird_rect.bottom>=900:
        death_sound.play()
        return False
    
    return True

def rotate_bird(bird):
    new_bird= py.transform.rotozoom(bird,-bird_movement * 3,1)
    return new_bird
            
def bird_animation():
    new_bird= bird_frames[bird_index]
    new_bird_rect= new_bird.get_rect(center=(100,bird_rect.centery))
    return new_bird,new_bird_rect

def score_display(game_state):
    if game_state == 'main_game':
        score_surface= game_font.render(f'Score: {int(score)}',True,(255,255,255))
        score_rect=score_surface.get_rect(center=(288,100))
        screen.blit(score_surface,score_rect)
    if game_state == 'game_over':
        score_surface= game_font.render(f'Score: {int(score)}',True,(255,255,255))
        score_rect=score_surface.get_rect(center=(288,100))
        screen.blit(score_surface,score_rect)
        
        high_score_surface= game_font.render(f'High Score: {int(high_score)}',True,(255,255,255))
        high_score_rect=high_score_surface.get_rect(center=(288,850))
        screen.blit(high_score_surface,high_score_rect)
        
def update_score(score,high_score):
    if score > high_score:
        high_score=score
    return high_score

#py.mixer.pre_init(frequency=44100, size= 16, channels=1, buffer=256)
py.init()
screen= py.display.set_mode((576,1024))
clock=py.time.Clock()
game_font=py.font.Font('04B_19.ttf',40)

#Game variables
gravity = 0.25
bird_movement = 0
game_active= True
score= 0
high_score= 0

bg_surface=py.image.load("assets/background-day.png").convert()
bg_surface=py.transform.scale2x(bg_surface)

floor_surface=py.image.load('assets/base.png').convert()
floor_surface=py.transform.scale2x(floor_surface)
floor_x_pos=0

    
# bird_surface= py.image.load('assets/bluebird-midflap.png').convert_alpha()
# bird_surface=py.transform.scale2x(bird_surface)
# bird_rect=bird_surface.get_rect(center = (100,512))

bird_downflap=py.transform.scale2x(py.image.load('assets/bluebird-downflap.png').convert_alpha())
bird_midflap=py.transform.scale2x(py.image.load('assets/bluebird-midflap.png').convert_alpha())
bird_upflap=py.transform.scale2x(py.image.load('assets/bluebird-upflap.png').convert_alpha())
bird_frames=[bird_downflap,bird_midflap,bird_upflap]
bird_index=0
bird_surface=bird_frames[bird_index]
bird_rect=bird_surface.get_rect(center=(100,512))

BIRDFLAP= py.USEREVENT+1
py.time.set_timer(BIRDFLAP,200)

pipe_surface=py.image.load('assets/pipe-green.png').convert()
pipe_surface=py.transform.scale2x(pipe_surface)
pipe_list=[]
SPAWNPIPE = py.USEREVENT
py.time.set_timer(SPAWNPIPE,1200)
pipe_height=[400,600,800]

game_over_surface=py.transform.scale2x(py.image.load('assets/message.png').convert_alpha())
game_over_rect= game_over_surface.get_rect(center=(288,512))

flap_sound=py.mixer.Sound('sound/sfx_wing.wav')
death_sound= py.mixer.Sound('sound/sfx_hit.wav')
score_sound= py.mixer.Sound('sound/sfx_point.wav')
score_sound_countdown =100

while True:
    #image of player1
    #background image
    
    for event in py.event.get():
        if event.type== py.QUIT :
            py.quit()
            sys.exit()
        if event.type== py.KEYDOWN:
            if event.key == py.K_SPACE and game_active:
                bird_movement=0
                bird_movement-= 12
                flap_sound.play()
            if event.key == py.K_SPACE and game_active == False:
                game_active= True
                pipe_list.clear()
                bird_rect.center= (100,512)
                bird_movement = 0
                score=0
                
        if event.type== SPAWNPIPE:
            pipe_list.extend(create_pipe())
        
        if event.type == BIRDFLAP:
            if bird_index <2:
                bird_index += 1
            else:
                bird_index=0
            
            bird_surface,bird_rect= bird_animation()
            

        
    screen.blit(bg_surface,(0,0))
    
    if game_active:
        # bird movement
        bird_movement+= gravity
        rotated_bird = rotate_bird(bird_surface)
        bird_rect.centery+= bird_movement
        screen.blit(rotated_bird,bird_rect)
        game_active=check_collision(pipe_list)
    
    #pipe movement
        pipe_list=move_pipes(pipe_list)
        draw_pipes(pipe_list)
        score += 0.01
        score_display('main_game')
        score_sound_countdown -= 1
        if score_sound_countdown<=0:
            score_sound.play()
            score_sound_countdown=100
            
    else:
        screen.blit(game_over_surface,game_over_rect)
        high_score=update_score(score,high_score)
        score_display('game_over')
    
    floor_x_pos-= 1
    drawfloor()
    if(floor_x_pos <= -576):
        floor_x_pos=0  
        
    screen.blit(floor_surface,(floor_x_pos,900))
    py.display.update()
    clock.tick(120)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\anand\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
